In [1]:
import json
import pandas as pd
from pathlib import Path
from pprint import pprint
import datetime
import time
import numpy as np
import matplotlib as plt

import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

%matplotlib inline

In [2]:
#path_results = Path('/media/discoD/models/elmo/ner/results_20_epochs_ibm')
#path_results = Path('/media/discoD/models/elmo/ner/results_1_epoch_harem_all_combinations')
# path_results = Path('/media/discoD/models/elmo/ner/mestrado/results_harem_final/')
# path_results = Path('/media/discoD/models/elmo/ner/mestrado/results_datalawyer_final_3.1/')
path_results = Path('/media/discoD/models/elmo/ner/mestrado/results_datalawyer_final_3.3/')

In [3]:
for folder in path_results.iterdir():
    print(folder)

/media/discoD/models/elmo/ner/mestrado/results_datalawyer_final_3.3/datalawyer-ft_ELMo+GloVe
/media/discoD/models/elmo/ner/mestrado/results_datalawyer_final_3.3/datalawyer-ft_ELMo+Word2Vec-jur


In [4]:
configs = dict()
metrics = dict()
for model_folder in path_results.iterdir():
#     print('Reading embeddings folder from %s' % model_folder.name)
    for embedding_folder in model_folder.iterdir():
#         print('Reading files from %s' % embedding_folder.name)
        key = embedding_folder.name
#         print('Reading files from %s' % key)
        has_metrics = False
        for results_file in embedding_folder.iterdir():
            if results_file.name.endswith('.json'):
                #print('Parsing data from %s' % results_file.name)
                if results_file.name.startswith('config'):
                    configs[key] = json.loads(results_file.read_bytes())
                elif results_file.name.startswith('metrics.'):
                    metrics[key] = json.loads(results_file.read_bytes())
                    has_metrics = True
        if not has_metrics:
            print('metrics.json not found for %s' % key)
            del configs[key]
    
print(len(metrics))
print(len(configs))

metrics.json not found for datalawyer-ft_ELMo+Word2Vec-jur_2
4
4


In [5]:
def get_seconds(time_str):
    x = time.strptime(time_str.split('.')[0],'%H:%M:%S')
    return datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
def get_average_epoch_duration(metrics):
    seconds = get_seconds(metrics['training_duration'])
    training_epochs = metrics['training_epochs'] + 1
    return seconds / training_epochs

In [6]:
TRAINING_ID = 'Training_ID'
MODEL = 'Model'
REPRESENTATION = 'Representation'
EMBEDDING = 'Embedding'
EMBEDDING_TYPE = 'Embedding_Type'
BATCH_SIZE = 'Batch_Size'
ELMO_DROPOUT = 'ELMo_Dropout'
TRAINING_NUMBER = 'Training_Number'
EXECUTION_NUMBER = 'Execution_Number'
BEST_EPOCH = 'Best_Epoch'
TRAINING_EPOCHS = 'Training_Epochs'
TRAINING_DURATION = 'Training_Duration'
TOTAL_DURATION = 'Total_Duration(s)'
AVERAGE_EPOCH_DURATION = 'Average_Epoch_Duration(s)'
TRAINING_ACCURACY = 'Training_Accuracy'
TRAINING_ACCURACY_TOP3 = 'Training_Accuracy_Top-3'
TRAINING_PRECISION = 'Training_Precision'
TRAINING_RECALL = 'Training_Recall'
TRAINING_F1_MEASURE = 'Training_F1-Measure'
TRAINING_LOSS = 'Training_Loss'
BEST_VALIDATION_ACCURACY = 'Best_Validation_Accuracy'
BEST_VALIDATION_ACCURACY_TOP3 = 'Best_Validation_Accuracy_Top-3'
BEST_VALIDATION_PRECISION = 'Best_Validation_Precision'
BEST_VALIDATION_RECALL = 'Best_Validation_Recall'
BEST_VALIDATION_F1_MEASURE = 'Best_Validation_F1-Measure'
BEST_VALIDATION_LOSS = 'Best_Validation_Loss'
TEST_ACCURACY = 'Test_Accuracy'
TEST_ACCURACY_TOP3 = 'Test_Accuracy_Top-3'
TEST_PRECISION = 'Test_Precision'
TEST_RECALL = 'Test_Recall'
TEST_F1_MEASURE = 'Test_F1_Measure'
TEST_LOSS = 'Test_Loss'
columns = [TRAINING_ID, MODEL, REPRESENTATION, EMBEDDING, EMBEDDING_TYPE, TRAINING_NUMBER, EXECUTION_NUMBER, BEST_EPOCH, TRAINING_EPOCHS, TRAINING_DURATION, TOTAL_DURATION, AVERAGE_EPOCH_DURATION, TRAINING_ACCURACY, TRAINING_ACCURACY_TOP3, TRAINING_PRECISION, TRAINING_RECALL, TRAINING_F1_MEASURE, TRAINING_LOSS, BEST_VALIDATION_ACCURACY, BEST_VALIDATION_ACCURACY_TOP3, BEST_VALIDATION_PRECISION, BEST_VALIDATION_RECALL, BEST_VALIDATION_F1_MEASURE, BEST_VALIDATION_LOSS, TEST_ACCURACY, TEST_ACCURACY_TOP3, TEST_PRECISION, TEST_RECALL, TEST_F1_MEASURE, TEST_LOSS]
columns

['Training_ID',
 'Model',
 'Representation',
 'Embedding',
 'Embedding_Type',
 'Training_Number',
 'Execution_Number',
 'Best_Epoch',
 'Training_Epochs',
 'Training_Duration',
 'Total_Duration(s)',
 'Average_Epoch_Duration(s)',
 'Training_Accuracy',
 'Training_Accuracy_Top-3',
 'Training_Precision',
 'Training_Recall',
 'Training_F1-Measure',
 'Training_Loss',
 'Best_Validation_Accuracy',
 'Best_Validation_Accuracy_Top-3',
 'Best_Validation_Precision',
 'Best_Validation_Recall',
 'Best_Validation_F1-Measure',
 'Best_Validation_Loss',
 'Test_Accuracy',
 'Test_Accuracy_Top-3',
 'Test_Precision',
 'Test_Recall',
 'Test_F1_Measure',
 'Test_Loss']

In [7]:
def get_training_data_from_id(training_id, config):
    data = training_id.split('_')
    print(data)
    if len(data) == 3:
        return {MODEL: data[0], REPRESENTATION: data[1], EXECUTION_NUMBER: data[2]}
    else:
        print('Check id: %s' % training_id)

In [8]:
training_data = []
for training_id, config in configs.items():
    training_metrics = metrics[training_id]
    data = get_training_data_from_id(training_id, config)
    data[TRAINING_ID] = training_id
    data[BEST_EPOCH] = training_metrics['best_epoch'] if training_metrics else None
    data[TRAINING_EPOCHS] = training_metrics['training_epochs'] + 1 if training_metrics else None
    data[TRAINING_DURATION] = training_metrics['training_duration'] if training_metrics else None
    data[TOTAL_DURATION] = get_seconds(training_metrics['training_duration']) if training_metrics else None
    data[AVERAGE_EPOCH_DURATION] = get_average_epoch_duration(training_metrics) if training_metrics else None
    data[TRAINING_ACCURACY] = training_metrics['training_accuracy'] if training_metrics else None
    data[TRAINING_ACCURACY_TOP3] = training_metrics['training_accuracy3'] if training_metrics else None
    data[TRAINING_PRECISION] = training_metrics['training_precision-overall'] if training_metrics else None
    data[TRAINING_RECALL] = training_metrics['training_recall-overall'] if training_metrics else None
    data[TRAINING_F1_MEASURE] = training_metrics['training_f1-measure-overall'] if training_metrics else None
    data[TRAINING_LOSS] = training_metrics['training_loss'] if training_metrics else None
    data[BEST_VALIDATION_ACCURACY] = training_metrics['best_validation_accuracy'] if training_metrics else None
    data[BEST_VALIDATION_ACCURACY_TOP3] = training_metrics['best_validation_accuracy3'] if training_metrics else None
    data[BEST_VALIDATION_PRECISION] = training_metrics['best_validation_precision-overall'] if training_metrics else None
    data[BEST_VALIDATION_RECALL] = training_metrics['best_validation_recall-overall'] if training_metrics else None
    data[BEST_VALIDATION_F1_MEASURE] = training_metrics['best_validation_f1-measure-overall'] if training_metrics else None
    data[BEST_VALIDATION_LOSS] = training_metrics['best_validation_loss'] if training_metrics else None
    data[TEST_ACCURACY] = training_metrics['test_accuracy'] if training_metrics else None
    data[TEST_ACCURACY_TOP3] = training_metrics['test_accuracy3'] if training_metrics else None
    data[TEST_PRECISION] = training_metrics['test_precision-overall'] if training_metrics else None
    data[TEST_RECALL] = training_metrics['test_recall-overall'] if training_metrics else None
    data[TEST_F1_MEASURE] = training_metrics['test_f1-measure-overall'] if training_metrics else None
    data[TEST_LOSS] = training_metrics['test_loss'] if training_metrics else None
    training_data.append(data)

['datalawyer-ft', 'ELMo+GloVe', '0']
['datalawyer-ft', 'ELMo+GloVe', '1']
['datalawyer-ft', 'ELMo+Word2Vec-jur', '0']
['datalawyer-ft', 'ELMo+Word2Vec-jur', '1']


In [9]:
pd.set_option('display.max_columns', 1000)
training_data_df = pd.DataFrame(training_data, columns=columns)
# training_data_df = training_data_df[(training_data_df[EXECUTION_NUMBER]).astype(int) == 0]
training_data_df 

,Training_ID,Model,Representation,Embedding,Embedding_Type,Training_Number,Execution_Number,Best_Epoch,Training_Epochs,Training_Duration,Total_Duration(s),Average_Epoch_Duration(s),Training_Accuracy,Training_Accuracy_Top-3,Training_Precision,Training_Recall,Training_F1-Measure,Training_Loss,Best_Validation_Accuracy,Best_Validation_Accuracy_Top-3,Best_Validation_Precision,Best_Validation_Recall,Best_Validation_F1-Measure,Best_Validation_Loss,Test_Accuracy,Test_Accuracy_Top-3,Test_Precision,Test_Recall,Test_F1_Measure,Test_Loss
0,datalawyer-ft_ELMo+GloVe_0,datalawyer-ft,ELMo+GloVe,NaN,NaN,NaN,0,23,48,3:55:45.173863,14145.0,294.687500,0.998929,0.998969,0.989175,0.988720,0.988948,1.037505,0.992349,0.992575,0.936932,0.946483,0.941684,17.482031,0.991801,0.991921,0.930888,0.928420,0.929652,19.878473
1,datalawyer-ft_ELMo+GloVe_1,datalawyer-ft,ELMo+GloVe,NaN,NaN,NaN,1,46,50,4:06:22.970549,14782.0,295.640000,0.998908,0.998929,0.988839,0.989180,0.989010,1.159503,0.992801,0.993055,0.939625,0.943935,0.941775,19.320777,0.993070,0.993444,0.937700,0.933722,0.935707,20.083548
2,datalawyer-ft_ELMo+Word2Vec-jur_0,datalawyer-ft,ELMo+Word2Vec-jur,NaN,NaN,NaN,0,12,37,3:03:51.427517,11031.0,298.135135,0.998750,0.998793,0.986766,0.986994,0.986880,1.351021,0.993380,0.993676,0.940283,0.946993,0.943626,14.378433,0.992174,0.992428,0.929104,0.924178,0.926635,17.243205
3,datalawyer-ft_ELMo+Word2Vec-jur_1,datalawyer-ft,ELMo+Word2Vec-jur,NaN,NaN,NaN,1,34,50,4:08:07.921361,14887.0,297.740000,0.999042,0.999057,0.990000,0.991367,0.990683,1.129547,0.993027,0.993267,0.940041,0.942915,0.941476,16.747036,0.991607,0.991831,0.933725,0.926299,0.929997,21.752093


In [10]:
for i in range(10):
    print(len(training_data_df[(training_data_df[EXECUTION_NUMBER]).astype(int) == i]))

2
2
0
0
0
0
0
0
0
0


In [11]:
training_data_df.to_csv('training_data_50_epochs_datalawyer_final.csv', index=None)

In [12]:
str(datetime.timedelta(seconds=training_data_df[TOTAL_DURATION].mean()))

'3:48:31.250000'

In [13]:
default_exported_columns = ['mean', 'min', 'max', 'std']
renamed_columns = ['Contagem', 'F-Score', 'Desvio Padrão', 'Mínimo', '25%', '50%', '75%', 'Máximo']
exported_columns_names = ['F-Score', 'Mínimo', 'Máximo', 'Desvio Padrão']
model_map = {'harem-ft': 'Sim', 'harem': 'Não'}
representation_map = {'ELMo+CNN+Embeddings': 'ELMo+CNN+Vetor', 'ELMo+Embeddings': 'ELMo+Vetor'}
embedding_type_map = {'skip': 'Skip-Gram', 'No': 'Sem Vetor', 'cbow': 'CBoW'}
embedding_map = {'wang2vec': 'Wang2Vec', 'glove': 'GloVe', 'word2vec': 'Word2Vec', 'No': 'Sem Vetor', 'fasttext': 'FastText'}
replacements_map = {**model_map, **representation_map, **embedding_type_map, **embedding_map}

def get_group_csv(index_names, group_name, group_columns, target_value, index_map, dataframe=training_data_df, exported_columns=default_exported_columns):
    if type(index_names) == str:
        index_names = [index_names]
    group = dataframe.groupby(group_columns).describe()[target_value].sort_values(by='mean', ascending=False)
    for column in exported_columns:
        group[column] = group[column] * 100
    group.index.names = index_names
    group = group.rename(index=index_map)
    group.columns = renamed_columns
    group.to_csv('grupo_' + group_name + '_50_epochs_mestrado_datalawyer_final.csv', columns=exported_columns_names, float_format = '%.2f%%')
    return group

In [14]:
get_group_csv('Representação', REPRESENTATION, REPRESENTATION, TEST_F1_MEASURE, replacements_map)

,Contagem,F-Score,Desvio Padrão,Mínimo,25%,50%,75%,Máximo
Representação,,,,,,,,
ELMo+GloVe,2.0,93.267947,0.428114,92.965224,0.931166,0.932679,0.934193,93.570670
ELMo+Word2Vec-jur,2.0,92.831605,0.237770,92.663477,0.927475,0.928316,0.929157,92.999734


In [15]:
scores_path = Path('/media/discoD/models/elmo/ner/mestrado/scores_final')

In [16]:
scores_wang2vec_selective, scores_wang2vec_total, scores_glove_selective, scores_glove_total = [], [], [], []
for score_file in scores_path.iterdir():
    score = score_file.open(mode='r', encoding='utf8').readlines()[1].split()[-1]
    if 'Wang2Vec' in score_file.name:
        if 'selective' in score_file.name:
            scores_wang2vec_selective.append(float(score))
        elif 'total' in score_file.name:
            scores_wang2vec_total.append(float(score))
    elif 'GloVe' in score_file.name:
        if 'selective' in score_file.name:
            scores_glove_selective.append(float(score))
        elif 'total' in score_file.name:
            scores_glove_total.append(float(score))
print(len(scores_wang2vec_selective))
print(len(scores_wang2vec_total))
print(len(scores_glove_selective))
print(len(scores_glove_total))

10
10
10
10


In [17]:
import statistics

In [18]:
print(statistics.mean(scores_wang2vec_selective))
print(statistics.mean(scores_wang2vec_total))
print(statistics.mean(scores_glove_selective))
print(statistics.mean(scores_glove_total))

83.222
78.037
82.886
77.631


In [19]:
scores_datalawyer_path = Path('/media/discoD/models/elmo/ner/mestrado/scores_final_datalawyer')

In [20]:
scores_word2vec, scores_glove = [], []
for score_file in scores_datalawyer_path.iterdir():
    score = score_file.open(mode='r', encoding='utf8').readlines()[1].split()[-1]
    if 'Word2Vec' in score_file.name:
        scores_word2vec.append(float(score))
    elif 'GloVe' in score_file.name:
        scores_glove.append(float(score))
print(len(scores_word2vec))
print(len(scores_glove))

2
2


In [21]:
print(statistics.mean(scores_word2vec))
print(statistics.mean(scores_glove))

93.405
93.81


In [22]:
print(statistics.stdev(scores_word2vec))
print(statistics.stdev(scores_glove))

0.38890872965260914
0.3535533905932738
